# Imports and Initializations

In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel, get_scheduler
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
from torch.cuda.amp import autocast, GradScaler
torch.backends.cuda.matmul.allow_tf32 = True
import pandas as pd

num_epochs = 2
lr = 5e-5                                           
batch_size = 1
warmup_steps= 750

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


# Initialising model

In [2]:
model = GPT2LMHeadModel.from_pretrained("gpt2-large")
tokenizer = AutoTokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

optimizer = AdamW(model.parameters(), lr=lr)

# Dataset

In [3]:
# dataset = load_dataset("MuskumPillerum/General-Knowledge")
# dataset = dataset['train']

In [4]:
# new_dataset = {'input_ids': [], 'attention_mask': []}

# data_format = 'Question:' + 'Answer: '
# ct = 0
# for example in dataset:
#     try:
#         input_text =  'Question:'+ example['Question']+ ' Answer: ' + example['Answer']
#         encoded_data = tokenizer('' + input_text + '', truncation=True, max_length=768, padding="max_length")
#         new_dataset['input_ids'].append(encoded_data['input_ids'])
#         new_dataset['attention_mask'].append(encoded_data['attention_mask'])
#     except:
#         ct += 1

# new_dataset = Dataset.from_dict(new_dataset)
# new_dataset.set_format("torch")

# # DataLoader
# dataloader = DataLoader(new_dataset, shuffle=True, batch_size=batch_size, pin_memory=True)
# ct

In [5]:
dataset = pd.read_csv('malayalam_dataset.csv')['Text']
dataset

0          മറ്റുള്ളവരെ കുറിച്ചു ചിന്തിക്കുന്നത്‌ മറ്റൊരു ...
1          ഓസ്ട്രേലിയൻ സർവീസ് ടീമിനെതിരെ ബാറ്റ് ചെയ്ത ബ്ര...
2                                          പുറത്ത്‌ മഞ്ഞുമഴ.
3                                 അതിന് ശേഷം ഇലക്ട്രിക് ബസ്.
4          മലയാളം, കന്നഡ, തമിഴ് തുടങ്ങിയ ഭാഷകളിൽ ശ്രദ്ധേയ...
                                 ...                        
6050951                      മദ്യപാനം ശരീരത്തിന് ഹാനികരമാണ്.
6050952                                  അങ്ങേര് നല്ല ഉശിരൻ.
6050953    നെല്ലും വാഴയും പച്ചക്കറികളും മരച്ചീനിയുമുൾപ്പെ...
6050954                           എളുപ്പത്തില്‍ അടുക്കാറില്ല
6050955    സംഭവം കൊലപാതകമാണെന്ന് സംശയിക്കുന്നതായി പൊലീസ് ...
Name: Text, Length: 6050956, dtype: object

In [ ]:
ocw

In [6]:
new_dataset = {'input_ids': [], 'attention_mask': []}
wrong_data = 0
for i,data in enumerate(dataset):
    if i%1e5: print()
    try:
        encoded_data = tokenizer(data+'', truncation=True, max_length=768, padding="max_length")
        new_dataset['input_ids'].append(encoded_data['input_ids'])
        new_dataset['attention_mask'].append(encoded_data['attention_mask'])
    except:
        wrong_data += 1
wrong_data

In [ ]:
new_dataset = Dataset.from_dict(new_dataset)
new_dataset.set_format("torch")
dataloader = DataLoader(new_dataset, shuffle=True, batch_size=batch_size, pin_memory=True)


In [ ]:
cow

# Inits

In [ ]:
num_training_steps = num_epochs * len(dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=warmup_steps, num_training_steps=num_training_steps)
model.resize_token_embeddings(len(tokenizer))
model.to(device)
model = torch.compile(model)
scaler = GradScaler()

# Training loop

In [ ]:
save_loc = '/home/arjun/Desktop/Model_save'

In [ ]:
progress_bar = tqdm(range(num_training_steps-1), desc='Training', unit='steps')
model.train()
ep = 0
prev_avg_train_loss = 999
for epoch in range(num_epochs):
    total_train_loss = 0
    for batch in dataloader:
        batch_data = batch['input_ids'].to(device)
        attention = batch['attention_mask'].to(device)

        optimizer.zero_grad()

        with autocast():
            outputs = model(batch_data,
                            labels=batch_data,
                            attention_mask=attention,
                            token_type_ids=None
                            )

            loss = outputs[0]
            batch_loss = loss.item()
            total_train_loss += batch_loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        lr_scheduler.step()
        progress_bar.update(1)

    avg_train_loss = total_train_loss / len(dataloader)
    ep += 1
    print('Epoch:', ep, 'Average training loss =', avg_train_loss)
    if abs(prev_avg_train_loss - avg_train_loss) < 0.0001:
        model.save_pretrained(save_loc)
        tokenizer.save_pretrained(save_loc)
        print("Loss is very small")
        break
    prev_avg_train_loss = avg_train_loss
    model.save_pretrained(save_loc)
    tokenizer.save_pretrained(save_loc)
